In [1]:
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 6857.02it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4297.68it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 234263/234263 [00:00<00:00, 2593163.08it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8161.95it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4133.19it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 106807.41it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8129.53it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 3435.25it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 9195.61it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 5578.82it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4034.12it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 107222.32it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.MatchPyramid()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = True
model.params['num_blocks'] = 2
model.params['kernel_count'] = [16, 32]
model.params['kernel_size'] = [[3, 3], [3, 3]]
model.params['dpool_size'] = [3, 10]
model.params['optimizer'] = 'adam'
model.params['dropout_rate'] = 0.1
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             4963800     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
matching_layer_1 (MatchingLayer (None, 10, 40, 1)    0           embedding[0][0]                  
          

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
train_generator = mz.DPoolPairDataGenerator(train_pack_processed,
                                            fixed_length_left=10,
                                            fixed_length_right=40,
                                            num_dup=2,
                                            num_neg=1,
                                            batch_size=20)
len(train_generator)

102

In [10]:
predict_generator = mz.DPoolDataGenerator(predict_pack_processed,
                                          fixed_length_left=10,
                                          fixed_length_right=40,
                                          batch_size=20)
len(predict_generator)

118

In [11]:
pred_x, pred_y = predict_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y))

In [12]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/20
102/102 [==============================] - 6s 60ms/step - loss: 0.7690
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5692250932507502 - normalized_discounted_cumulative_gain@5(0.0): 0.6195008780055599 - mean_average_precision(0.0): 0.5879674081412779
Epoch 2/20
102/102 [==============================] - 7s 73ms/step - loss: 0.4052
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5577580076131701 - normalized_discounted_cumulative_gain@5(0.0): 0.6208268248908941 - mean_average_precision(0.0): 0.5803710108112564
Epoch 3/20
102/102 [==============================] - 8s 82ms/step - loss: 0.2382
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5374999946557393 - normalized_discounted_cumulative_gain@5(0.0): 0.5998451250432698 - mean_average_precision(0.0): 0.5588963866653741
Epoch 4/20
102/102 [==============================] - 5s 54ms/step - loss: 0.1180
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5785198780057831 - normalized_

In [17]:
model.evaluate(pred_x, pred_y, batch_size=5)

{normalized_discounted_cumulative_gain@3(0.0): 0.5486963704637519,
 normalized_discounted_cumulative_gain@5(0.0): 0.605962157344213,
 mean_average_precision(0.0): 0.5658641479427621}